##  Code runs, but does not have ClEAR Value Label and service catalog whos up duplicate instance for the opportunity items.

In [6]:
import pandas as pd
from datetime import datetime

# Function to format revenue and date
def format_revenue_and_date(revenue, date):
    revenue_text = f" - ${revenue:,.0f}" if pd.notnull(revenue) else ""
    date_text = f" - {date.strftime('%m/%d/%Y')}" if pd.notnull(date) else ""
    return revenue_text + date_text

# Function to calculate 'CLEAR Value'
def calculate_clear_value(transactions):
    total_revenue = transactions['Revenue'].sum()
    last_transaction_date = transactions['Transaction Date'].max()
    divisions = transactions['Division'].unique()
    
    if len(divisions) == 0:
        return "Opportunity"
    elif len(divisions) > 1:
        service_text = "Both"
    else:
        service_text = divisions[0]
        
    return service_text + format_revenue_and_date(total_revenue, last_transaction_date)

# Load transactional data and service catalog
transactional_data_path = r'C:\Users\proto\OneDrive\ConWork\synthetic_project_clear_data_DEV.xlsx'  # Update with the actual path
service_catalog_path = r'C:\Users\proto\OneDrive\ConWork\service_catalog.txt'  # Update with the actual path

transactions_df = pd.read_excel(transactional_data_path)
service_catalog_df = pd.read_csv(service_catalog_path, delimiter='\t')

# Identifying unique combinations of service categories and subservice categories from the service catalog
valid_combinations = service_catalog_df.drop_duplicates(subset=['Service Category', 'SubService Category'])

# Identify unique services provided to each client and calculate 'CLEAR Value'
unique_services = transactions_df.groupby(['Client Name', 'Service Category', 'SubService Category']).apply(calculate_clear_value).reset_index(name='CLEAR Value')

# Join the 'CLEAR Value' back to the transactional data
enhanced_transactions_df = pd.merge(transactions_df, unique_services, on=['Client Name', 'Service Category', 'SubService Category'], how='left')

# Create a DataFrame of all possible combinations of clients and VALID service catalog entries
clients = transactions_df['Client Name'].unique()
all_possible_services = pd.MultiIndex.from_product([clients, valid_combinations['Service Category'], valid_combinations['SubService Category']], names=['Client Name', 'Service Category', 'SubService Category']).to_frame(index=False)

# Filter out combinations that don't exist in the service catalog
valid_service_combinations = all_possible_services.merge(valid_combinations, on=['Service Category', 'SubService Category'], how='inner')

# Find services from the catalog not provided to each client and mark as 'Opportunity'
opportunities = pd.merge(valid_service_combinations, unique_services, on=['Client Name', 'Service Category', 'SubService Category'], how='left', indicator=True)
opportunities = opportunities[opportunities['_merge'] == 'left_only'].drop(columns='_merge')
opportunities['CLEAR Value'] = "Opportunity"

# Combine provided services with opportunities
final_df = pd.concat([enhanced_transactions_df, opportunities])

# Add missing columns for opportunities with NaN values
missing_cols = set(transactions_df.columns) - set(final_df.columns)
for col in missing_cols:
    final_df[col] = pd.NA

# Rearrange columns to match the original transactional data
final_df = final_df[transactions_df.columns.tolist() + ['CLEAR Value']]

# Current timestamp to append to the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_output_path = f'C:\\Users\\proto\\OneDrive\\ConWork\\final_output_{timestamp}.csv'

# Output the final DataFrame to a CSV file
final_df.to_csv(final_output_path, index=False)


## This Code works well

In [5]:
import pandas as pd
from datetime import datetime

# Function to calculate 'CLEAR Value Label'
def calculate_clear_value_label(transactions):
    divisions = transactions['Division'].unique()
    if len(divisions) == 0:
        return "Opportunity"
    elif len(divisions) > 1:
        return "Both"
    else:
        return divisions[0]

# Function to format revenue and date
def format_revenue_and_date(revenue, date):
    revenue_text = f" - ${revenue:,.0f}" if pd.notnull(revenue) else ""
    date_text = f" - {date.strftime('%m/%d/%Y')}" if pd.notnull(date) else ""
    return revenue_text + date_text

# Load transactional data and service catalog
transactional_data_path = r'C:\Users\proto\OneDrive\ConWork\synthetic_project_clear_data_DEV.xlsx'  # Update with the actual path
service_catalog_path = r'C:\Users\proto\OneDrive\ConWork\service_catalog.txt'  # Update with the actual path

transactions_df = pd.read_excel(transactional_data_path)
service_catalog_df = pd.read_csv(service_catalog_path, delimiter='\t')

# Identifying unique combinations from the service catalog
valid_combinations = service_catalog_df.drop_duplicates(subset=['Service Category', 'SubService Category'])

# Calculate 'CLEAR Value Label' for each unique service provided
grouped = transactions_df.groupby(['Client Name', 'Service Category', 'SubService Category'])
unique_services_label = grouped.apply(calculate_clear_value_label).reset_index(name='CLEAR Value Label')

# Calculate 'CLEAR Value' for each unique service provided
def calculate_clear_value(row):
    transactions = transactions_df[(transactions_df['Client Name'] == row['Client Name']) & 
                                   (transactions_df['Service Category'] == row['Service Category']) & 
                                   (transactions_df['SubService Category'] == row['SubService Category'])]
    total_revenue = transactions['Revenue'].sum()
    last_transaction_date = transactions['Transaction Date'].max()
    return row['CLEAR Value Label'] + format_revenue_and_date(total_revenue, last_transaction_date)

unique_services_label['CLEAR Value'] = unique_services_label.apply(calculate_clear_value, axis=1)

# Create a DataFrame of all possible combinations of clients and VALID service catalog entries
clients = transactions_df['Client Name'].unique()
all_possible_services = pd.MultiIndex.from_product([clients, valid_combinations['Service Category'], valid_combinations['SubService Category']], names=['Client Name', 'Service Category', 'SubService Category']).to_frame(index=False)

# Find services from the catalog not provided to each client and mark as 'Opportunity'
opportunities = pd.merge(all_possible_services, unique_services_label, on=['Client Name', 'Service Category', 'SubService Category'], how='left', indicator=True)
opportunities = opportunities[opportunities['_merge'] == 'left_only'].drop(columns='_merge')
opportunities['CLEAR Value Label'] = "Opportunity"
opportunities['CLEAR Value'] = "Opportunity"

# Combine provided services with opportunities
final_df = pd.concat([unique_services_label, opportunities])

# Add missing columns for opportunities with NaN values
missing_cols = set(transactions_df.columns) - set(final_df.columns)
for col in missing_cols:
    final_df[col] = pd.NA

# Rearrange columns to match the original transactional data and include 'CLEAR Value Label' and 'CLEAR Value'
final_df = final_df[transactions_df.columns.tolist() + ['CLEAR Value Label', 'CLEAR Value']]

# Current timestamp to append to the file name and output the final DataFrame
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_output_path = f'C:\\Users\\proto\\OneDrive\\ConWork\\final_output_{timestamp}.csv'
final_df.to_csv(final_output_path, index=False)

KeyboardInterrupt: 

## Code restructured to run quicker, but needs attention to logic of non transactional items and fomatting.

In [3]:
import pandas as pd
from datetime import datetime

# Load transactional data and service catalog
transactional_data_path = r'C:\Users\proto\OneDrive\ConWork\synthetic_project_clear_data_DEV.xlsx'
service_catalog_path = r'C:\Users\proto\OneDrive\ConWork\service_catalog.txt'

transactions_df = pd.read_excel(transactional_data_path)
service_catalog_df = pd.read_csv(service_catalog_path, delimiter='\t')

# Vectorized calculation of 'CLEAR Value Label'
def vectorized_clear_value_label(df):
    df['CLEAR Value Label'] = df['Division']
    df.loc[df['Division'].duplicated(keep=False), 'CLEAR Value Label'] = 'Both'
    df['CLEAR Value Label'].fillna('Opportunity', inplace=True)
    return df

transactions_df = vectorized_clear_value_label(transactions_df)

# Function to format revenue and date
def format_revenue_and_date(revenue, date):
    revenue_text = f" - ${revenue:,.0f}" if pd.notnull(revenue) and revenue >= 0 else ""
    date_text = f" - {date.strftime('%m/%d/%Y')}" if pd.notnull(date) else ""
    return revenue_text + date_text

# Calculate 'CLEAR Value'
transactions_df['CLEAR Value'] = transactions_df.apply(lambda row: row['CLEAR Value Label'] + format_revenue_and_date(row['Revenue'], row['Transaction Date']), axis=1)

# Prepare for opportunities calculation
valid_combinations = service_catalog_df.drop_duplicates(subset=['Service Category', 'SubService Category'])
clients = transactions_df['Client Name'].unique()
all_possible_services = pd.MultiIndex.from_product([clients, valid_combinations['Service Category'], valid_combinations['SubService Category']], names=['Client Name', 'Service Category', 'SubService Category']).to_frame(index=False)

# Calculate opportunities
opportunities = pd.merge(all_possible_services, transactions_df[['Client Name', 'Service Category', 'SubService Category', 'CLEAR Value']], on=['Client Name', 'Service Category', 'SubService Category'], how='left', indicator=True)
opportunities = opportunities[opportunities['_merge'] == 'left_only'].drop(columns='_merge')
opportunities['CLEAR Value'] = "Opportunity"

# Combine provided services with opportunities
final_df = pd.concat([transactions_df, opportunities], ignore_index=True)

# Add missing columns for opportunities with NaN values
missing_cols = set(transactions_df.columns) - set(final_df.columns)
for col in missing_cols:
    final_df[col] = pd.NA

# Rearrange columns to match the original transactional data
final_df = final_df[transactions_df.columns.tolist() + ['CLEAR Value Label', 'CLEAR Value']]

# Current timestamp to append to the file name and output the final DataFrame
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_output_path = f'C:\\Users\\proto\\OneDrive\\ConWork\\final_output_{timestamp}.csv'
final_df.to_csv(final_output_path, index=False)
